<a href="https://colab.research.google.com/github/ibacaraujo/to-tensorflow2/blob/master/classify_structured_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classify Structured Data

In [1]:
!pip install sklearn

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd

!pip install -q tensorflow==2.0.0-beta1
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 87.9MB 1.2MB/s 
     |████████████████████████████████| 501kB 40.8MB/s 
     |████████████████████████████████| 3.1MB 32.8MB/s 


## Create dataframe

In [3]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv(URL)
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


## Split the dataframe

In [4]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'val examples')
print(len(test), 'test examples')

193 train examples
49 val examples
61 test examples


## Create an input pipeline using tf.data

tf.data to wrap the dataframe, so we will be able to use feature columns as a bridge to map from the columns in the Pandas dataframe to features used to train the model.

If it were a large CSV file, in a way that does not fit in memory, we would use tf.data to read it from disk directly. Interesting! To learn!

In [0]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [0]:
batch_size = 5 # small batch for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, batch_size=batch_size)
test_ds = df_to_dataset(test, batch_size=batch_size)

## Understand the input pipeline

Let's see the format of the input pipeline

In [8]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['age'])
  print('A batch of targets:', label_batch)

Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([58 55 63 57 45], shape=(5,), dtype=int32)
A batch of targets: tf.Tensor([0 0 1 0 0], shape=(5,), dtype=int32)


A dictionary with keys being the feature names and columns as tf.Tensor.

## Demonstrate several types of feature column

In [0]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

In [0]:
# A utility method to create a feature column and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [16]:
age = feature_column.numeric_column("age")
demo(age)

[[58.]
 [55.]
 [63.]
 [57.]
 [45.]]


In [18]:
age_buckets = feature_column.bucketized_column(age, 
  boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
demo(age_buckets)

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


In [19]:
thal = feature_column.categorical_column_with_vocabulary_list(
    'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)

W0808 03:06:27.011060 140159448045440 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:2655: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0808 03:06:27.023420 140159448045440 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4215: IndicatorColumn._variable_shape (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
W0808 03:06:27.025430 140159448045440 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4270: VocabularyListCateg

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [20]:
# embedding when the number of categories is high
thal_embedding = feature_column.embedding_column(thal, dimension=8)
demo(thal_embedding)

[[ 0.32926026  0.6037977   0.0859094  -0.1343561   0.18337654 -0.260106
  -0.09752115 -0.32585666]
 [-0.2929751  -0.55914295 -0.33032867 -0.26798528  0.05934702 -0.19060789
  -0.2030002  -0.05641369]
 [ 0.32926026  0.6037977   0.0859094  -0.1343561   0.18337654 -0.260106
  -0.09752115 -0.32585666]
 [ 0.32926026  0.6037977   0.0859094  -0.1343561   0.18337654 -0.260106
  -0.09752115 -0.32585666]
 [-0.2929751  -0.55914295 -0.33032867 -0.26798528  0.05934702 -0.19060789
  -0.2030002  -0.05641369]]


In [21]:
thal_hashed = feature_column.categorical_column_with_hash_bucket(
    'thal', hash_bucket_size=1000)
demo(feature_column.indicator_column(thal_hashed))

W0808 03:17:05.568101 140159448045440 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4270: HashedCategoricalColumn._num_buckets (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [22]:
cross_feature = feature_column.crossed_column(
    [age_buckets, thal], hash_bucket_size=1000)
demo(feature_column.indicator_column(cross_feature))

W0808 03:18:40.419500 140159448045440 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/feature_column/feature_column_v2.py:4270: CrossedColumn._num_buckets (from tensorflow.python.feature_column.feature_column_v2) is deprecated and will be removed in a future version.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Choose which columns to use

Let's use the feature columns to train a model.

Key point: if your aim is to build an accurate model, try a larger dataset of your own, and think carefully about which features are the most meaningful to include, and how they should be represented.

Great!

In [0]:
feature_columns = []

# numeric cols
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
  feature_columns.append(feature_column.numeric_column(header))
  
# bucketized cols
age_buckets = feature_column.bucketized_column(
    age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

# indicator cols
thal = feature_column.categorical_column_with_vocabulary_list(
  'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# embedding cols
thal_embedding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

# crossed cols
crossed_feature = feature_column.crossed_column(
    [age_buckets, thal], hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)

## Create a feature layer

So, we have defined our feature columns, now we will use tf.keras.DenseFeatures layer to input them to our Keras model.

In [0]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

Now, new input pipeline with larger batch size

In [0]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, batch_size=batch_size)
test_ds = df_to_dataset(test, batch_size=batch_size)

## Create, compile and train the model!

In [34]:
model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'],
             run_eagerly=True)

model.fit(train_ds,
         validation_data=val_ds,
         epochs=5)

Epoch 1/5
7/7 [==============================] - 1s 99ms/step - loss: 0.8140 - accuracy: 0.6729 - val_loss: 0.6456 - val_accuracy: 0.6122
Epoch 2/5
7/7 [==============================] - 0s 38ms/step - loss: 0.5108 - accuracy: 0.7628 - val_loss: 0.5694 - val_accuracy: 0.6939
Epoch 3/5
7/7 [==============================] - 0s 37ms/step - loss: 0.4330 - accuracy: 0.8160 - val_loss: 0.5498 - val_accuracy: 0.6735
Epoch 4/5
7/7 [==============================] - 0s 37ms/step - loss: 0.4072 - accuracy: 0.8040 - val_loss: 0.5385 - val_accuracy: 0.6735
Epoch 5/5
7/7 [==============================] - 0s 37ms/step - loss: 0.4144 - accuracy: 0.8147 - val_loss: 0.5359 - val_accuracy: 0.6735


In [35]:
loss, accuracy = model.evaluate(test_ds)
print('Accuracy:', accuracy)

2/2 [==============================] - 0s 22ms/step - loss: 0.5395 - accuracy: 0.7541
Accuracy: 0.75409836


Key point: Best results with deep learning with much larger and more complex datasets. When working with a small dataset like this one, we recommend using a decision tree or random forest as a strong baseline. Code to use as a starting point when working with own datasets in the future.

Next steps. To work with another dataset! Think carefully about which features to include in the model, and how they should be represented.